In [72]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### Creating MC questions for VFX captioning
Using ChatGPT 4o mini I will create a set of MC questions for each champion ability set.

In [170]:
from VFXTesting.lol_champ_reader import LoLChampReader

LoLChampReader().get_champ_ability('Amumu', 'W')

{'name': 'Despair',
 'effects': 'Toggle: Amumu cries a continuous pool of tears, dealing magic damage every 0.5 seconds to nearby enemies.',
 'cost': 8,
 'cooldown': 1,
 'targeting': 'Auto',
 'affects': 'Enemies',
 'spellshieldable': 'False',
 'resource': 'MANA_PER_SECOND',
 'damageType': 'MAGIC_DAMAGE',
 'spellEffects': 'aoedot',
 'notes': "Toggle abilities do not count as ability activations for the purposes of on-cast effects such as  Spellblade and triggering  Force Pulse's passive.\nDespite the rounding in Despair's tooltip, it will deal increased damage for every point of ability power.",
 'blurb': 'Toggle:  Amumu begins crying, continually dealing magic damage to nearby enemies based on their maximum health.',
 'castTime': 'none',
 'effectRadius': '350'}

In [94]:
from openai import OpenAI

MODEL = 'gpt-4o-mini'
API_KEY=os.getenv("OPENAI_KEY")

client = OpenAI(api_key=API_KEY)


In [165]:
champ = "Ahri"
ability = "W"

completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a League of Legends creating questions for testing ability VFX, do not mention the champion name or LoL terminology in the questions or answers, if asking questions about colors be sure to be correct"},
        {"role": "assistant", "content": str(LoLChampReader().get_champ_ability(champ, ability))},
        {"role": "assistant", "content": "JSON format: {['question': <question>, 'options': [<option_a>, <option_b>, <option_c>, <option_d>], 'correct_answer': <correct_answer>]}}"},
        {
            "role": "user",
            "content": f"Based on the {ability} ability description for {champ}, create 3 questions for the ability visuals, that each could be correctly answered using a single correctly-timed screenshot of the ability, reply only with the given JSON format"
        }
    ]
)

mc_questions = completion.choices[0].message

In [168]:
# json.loads(mc_questions.content.split('```')[1].replace('json', ''))
json.loads(mc_questions.content)

[{'question': 'What color do the flames appear when the ability is activated?',
  'options': ['Red', 'Blue', 'Green', 'Yellow'],
  'correct_answer': 'Red'},
 {'question': 'How many flames orbit the character after activating the ability?',
  'options': ['One', 'Two', 'Three', 'Four'],
  'correct_answer': 'Three'},
 {'question': 'What visual effect indicates that the flames are targeting an enemy?',
  'options': ['A glow around the flames',
   'A line connecting to the enemy',
   'A burst effect',
   'A color change'],
  'correct_answer': 'A line connecting to the enemy'}]

In [99]:
LoLChampReader().get_champ_ability(champ, ability)

{'name': 'Charm',
 'effects': 'Active: Ahri blows forth a kiss in the target direction that deals magic damage to the first enemy hit, knocking them down and charming and slowing them by 65% for a duration.',
 'cost': 60,
 'cooldown': 12,
 'targeting': 'Direction',
 'affects': 'Enemies',
 'spellshieldable': 'True',
 'resource': 'MANA',
 'damageType': 'MAGIC_DAMAGE',
 'spellEffects': 'spell',
 'projectile': 'TRUE',
 'notes': 'Charm is able to  slow targets otherwise  immune to slows upon  charming them.(bug)\nThis ability will cast from wherever the caster is at the end of the cast time.',
 'blurb': 'Active:  Ahri blows forth a kiss that  charms the first enemy hit, dealing magic damage and  knocking them down.',
 'speed': '1550',
 'width': '120',
 'castTime': '0.25'}